In [1]:
from build_dataset_dataloader import get_dataset

train_dataset, val_dataset, vocabulary = get_dataset(case=2)

/home/thangquang09/CODE/CTAI_MachineLearning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /home/thangquang09/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/thangquang09/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Kích thước từ vựng: 10000


In [2]:
from datasets import load_dataset

dataset = load_dataset("thangquang09/fake-new-imposter-hunt-in-texts") 

val = dataset['case2_validation'].to_pandas()
val['label'].value_counts()

label
1    59
2    41
Name: count, dtype: int64

In [7]:
from torch.utils.data import DataLoader
from CONFIG import *
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [3]:
from LSTM import TextClassificationLSTM
from CONFIG import *
import torch

model_path = "/home/thangquang09/CODE/CTAI_MachineLearning/models/TextClassificationLSTM_case2_best_pair_acc_20250824_185657.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextClassificationLSTM(
    vocab_size=len(vocabulary),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
)

model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()



TextClassificationLSTM(
  (embedding): Embedding(10000, 512, padding_idx=0)
  (lstm): LSTM(512, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (classifier): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=256, out_features=1, bias=True)
  )
  (embedding_dropout): Dropout(p=0.2, inplace=False)
)

In [5]:


def predict_pair_confidence(model, seq1, seq2):
    """
    Predict which text is REAL using confidence-based approach
    Returns: prediction (0 or 1), confidence score, explanation
    """
    model.eval()
    with torch.no_grad():
        # Get predictions for both texts
        logit1 = model(seq1)
        logit2 = model(seq2)
        
        # Convert to probabilities
        prob1 = torch.sigmoid(logit1)
        prob2 = torch.sigmoid(logit2)
        
        # Determine prediction based on confidence
        predictions = []
        explanations = []
        
        for i in range(len(prob1)):
            p1, p2 = prob1[i].item(), prob2[i].item()
            
            if p1 > 0.5 and p2 > 0.5:
                # Both predicted as REAL -> choose higher confidence
                if p1 > p2:
                    pred = 0  # text1 is REAL (label=1 in original format)
                    exp = f"Both REAL, text1 more confident ({p1:.4f} vs {p2:.4f})"
                else:
                    pred = 1  # text2 is REAL (label=2 in original format)
                    exp = f"Both REAL, text2 more confident ({p2:.4f} vs {p1:.4f})"
            elif p1 < 0.5 and p2 < 0.5:
                # Both predicted as FAKE -> choose less fake (higher prob)
                if p1 > p2:
                    pred = 0  # text1 less fake
                    exp = f"Both FAKE, text1 less fake ({p1:.4f} vs {p2:.4f})"
                else:
                    pred = 1  # text2 less fake
                    exp = f"Both FAKE, text2 less fake ({p2:.4f} vs {p1:.4f})"
            else:
                # Normal case: one REAL, one FAKE
                if p1 > 0.5:
                    pred = 0  # text1 is REAL
                    exp = f"Text1 REAL ({p1:.4f}), Text2 FAKE ({p2:.4f})"
                else:
                    pred = 1  # text2 is REAL
                    exp = f"Text2 REAL ({p2:.4f}), Text1 FAKE ({p1:.4f})"
            
            predictions.append(pred)
            explanations.append(exp)
        
        return torch.tensor(predictions, device=seq1.device), explanations


def evaluate_individual(model, valid_dataloader, criterion):
    """Evaluate on individual text classification"""
    model.eval()
    total_loss = 0
    running_correct = 0
    total = 0
    
    with torch.no_grad():
        for seq, labels in valid_dataloader:
            seq, labels = seq.to(device), labels.to(device).float()
            outputs = model(seq)
            loss = criterion(outputs.squeeze(1), labels)
            total_loss += loss.item()

            predicted = (torch.sigmoid(outputs.squeeze(1)) > 0.5).float()
            running_correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    accuracy = 100 * running_correct / total
    total_loss = total_loss / len(valid_dataloader)
    return total_loss, accuracy


def evaluate_pairs(model, pair_dataloader):
    """Evaluate on original pair comparison task"""
    model.eval()
    running_correct = 0
    total = 0
    total_predictions = []
    with torch.no_grad():
        for seq1, seq2, labels in pair_dataloader:
            seq1, seq2, labels = seq1.to(device), seq2.to(device), labels.to(device)
            
            # Get pair predictions using confidence approach
            predictions, _ = predict_pair_confidence(model, seq1, seq2)
            total_predictions.extend(predictions.cpu().numpy())
            # Convert original labels (0,1) to predictions format
            # Original: 0 means text1 is REAL, 1 means text2 is REAL
            running_correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    accuracy = 100 * running_correct / total
    return accuracy, total_predictions


In [8]:
acc, preds = evaluate_pairs(model, val_dataloader)

In [14]:
import numpy as np
np.unique_counts(preds)

UniqueCountsResult(values=array([0, 1]), counts=array([52, 48]))

In [6]:
from build_dataset_dataloader import TextComparisonDataset
from LSTM import TextClassificationLSTM
val_dataset, vocabulary = TextComparisonDataset(val)
from CONFIG import *

model = TextClassificationLSTM(
    vocab_size=len(vocabulary),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
)


TypeError: TextComparisonDataset.__init__() missing 1 required positional argument: 'vocab'

In [2]:
from build_dataset_dataloader import get_dataset, text_to_sequence, TextComparisonDataset

train_dataset, val_dataset, vocabulary = get_dataset(case=2)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/thangquang09/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/thangquang09/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Kích thước từ vựng: 9125
Index của '<pad>': 0
Index của một từ ngẫu nhiên 'hello': 2
Index của một từ không có trong từ điển: 2


In [2]:
len(vocabulary)

9125

In [3]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [4]:
next(iter(train_dataloader))

[tensor([[2620,    8, 2396,  ...,    0,    0,    0],
         [  26,  329,    4,  ...,    0,    0,    0],
         [  10,   14,    7,  ...,    0,    0,    0],
         ...,
         [   7,  308,   16,  ...,    0,    0,    0],
         [ 421,   81,   48,  ...,    0,    0,    0],
         [   3,   96,  116,  ...,    0,    0,    0]]),
 tensor([[   8, 2396,    4,  ...,    0,    0,    0],
         [  26,  329,    4,  ...,  571,  572,  573],
         [   7, 1285, 2626,  ...,  630, 1669,  807],
         ...,
         [6510,  467,  147,  ...,    0,    0,    0],
         [   7,  156, 1941,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]]),
 tensor([0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.])]

In [3]:
import torch
from torch import nn
from LSTM import SiameseLSTM

EMBEDDING_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = 1 # Output 1 giá trị logit cho binary classification
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
VOCAB_SIZE = len(vocabulary)
SEQ_LENGTH = 600

model = SiameseLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

NameError: name 'vocabulary' is not defined

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)


In [8]:
def evaluate(model, valid_dataloader, criterion):
    model.eval()
    total_loss = 0
    running_correct = 0
    total = 0
    with torch.no_grad():
        for seq1, seq2, labels in valid_dataloader:
            seq1, seq2, labels = seq1.to(device), seq2.to(device), labels.to(device).float()
            outputs = model(seq1, seq2)
            loss = criterion(outputs.squeeze(1), labels)
            total_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs.squeeze(1)) > 0.5).float()
            running_correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = 100 * running_correct / total
    total_loss = total_loss / total
    return total_loss, accuracy

In [9]:
import time

def train(model, max_epoch, train_dataloader, valid_dataloader, criterion, optimizer, device):
    model.to(device)
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    # callbacks (save best params)
    best_weights = None
    best_test_acc, best_test_loss = -1, float('inf')


    for epoch in range(max_epoch):
        model.train()
        running_loss = 0.0
        running_correct = 0   # to track number of correct predictions
        total = 0   
        
        for i, (seq1, seq2, labels) in enumerate(train_dataloader):
            epoch_start_time = time.time()
            
            seq1, seq2, labels = seq1.to(device), seq2.to(device), labels.to(device).float()
            
            optimizer.zero_grad()
            
            # Forward
            outputs = model(seq1, seq2)
            loss = criterion(outputs.squeeze(1), labels)
            running_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs.squeeze(1)) > 0.5).float()
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()
            
            # Backward
            loss.backward()
            optimizer.step()

        epoch_accuracy = 100 * running_correct / total
        epoch_loss = running_loss / total

        test_loss, test_accuracy = evaluate(model, valid_dataloader, criterion)

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            # best_test_acc = test_accuracy
            best_weights = model.state_dict()
            
        print(
            "| Epoch {:3d} | Time: {:5.2f}s | Train Accuracy {:8.3f}% | Train Loss {:8.3f} "
            "| Valid Accuracy {:8.3f}% | Valid Loss {:8.3f} ".format(
                epoch+1, time.time() - epoch_start_time, epoch_accuracy, epoch_loss, test_accuracy, test_loss
            )
        )
        
        # save for plot
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        
    values_dict = {
        "train_losses": train_losses,
        "train_accuracies": train_accuracies,
        "test_losses": test_losses,
        "test_accuracies": test_accuracies
    }

    return values_dict, best_weights

In [10]:
model.to(device)

history, best_weights = train(model, 2, train_dataloader, val_dataloader, criterion, optimizer, device)

| Epoch   1 | Time:  0.32s | Train Accuracy   58.068% | Train Loss    0.042 | Valid Accuracy   66.000% | Valid Loss    0.043 
| Epoch   2 | Time:  0.31s | Train Accuracy   60.895% | Train Loss    0.040 | Valid Accuracy   67.000% | Valid Loss    0.042 


## Make Submisison

In [6]:
import torch
from torch import nn
from LSTM import SiameseLSTM
from CONFIG import  *

# load model from /home/thangquang09/CODE/CTAI_MachineLearning/models/Siamese_LSTM_case2.pth
model = SiameseLSTM(
    vocab_size=len(vocabulary),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
)

model.load_state_dict(torch.load('/home/thangquang09/CODE/CTAI_MachineLearning/models/Siamese_LSTM_case2.pth', map_location='cpu'))


<All keys matched successfully>

In [7]:
import pandas as pd
from load_data import read_texts_from_dir
from torch.utils.data import DataLoader

df_test = read_texts_from_dir('/home/thangquang09/CODE/CTAI_MachineLearning/data/fake-or-real-the-impostor-hunt/data/test')
df_test['label'] = 3

test_dataset = TextComparisonDataset(df_test, vocabulary)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Number of directories: 1068


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
full_predicted = []

for seq1, seq2, labels in test_dataloader:
    seq1, seq2 = seq1.to(device), seq2.to(device)
    outputs = model(seq1, seq2)
    predicted = (torch.sigmoid(outputs.squeeze(1)) > 0.5).float()
    full_predicted.append(predicted)

# Concatenate all predictions
full_predicted = torch.cat(full_predicted, dim=0)


In [9]:
full_predicted = full_predicted.cpu().numpy()
full_predicted = full_predicted + 1

In [10]:

from pathlib import Path

def make_submission(y_pred, file_name):
    submission = pd.DataFrame({
        "id": df_test.index,
        "real_text_id": y_pred.astype(int)
    }).sort_values("id")

    save_path = Path(file_name)
    submission.to_csv(save_path, index=False)
    print(f"✅ Submission saved to {save_path.resolve()}")

In [11]:
make_submission(full_predicted, 'submission_case2_LSTM.csv')

✅ Submission saved to /home/thangquang09/CODE/CTAI_MachineLearning/src/submission_case2_LSTM.csv
